In [42]:
import pandas as pd
import numpy as np

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from stop_words import get_stop_words

import string
import re
from collections import Counter
from collections import OrderedDict
import unidecode
import requests

In [86]:
data = pd.read_json('gg2015.json')
data = pd.DataFrame(data)

In [87]:
data = data['text'].values.tolist()

In [88]:
#print(data[:10])

In [89]:
people = set()

In [104]:
url = 'https://query.wikidata.org/sparql'

In [156]:
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q2526255; #uncomment for     FILM director (no award for TV director)
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [163]:
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q10800557; #uncomment for    FILM actor (don't just use actor)
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r1 = requests.get(url, params = {'format': 'json', 'query': query})
kb1 = r1.json()
for item in kb1['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [164]:
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q10798782; #uncomment for    TV actor (don't just use actor)
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [165]:
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q36834; #uncomment for       composer (cannot use songwriter)
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [166]:
query = """
# ALL PERSONS required for awards
SELECT DISTINCT ?person ?personLabel WHERE {
# FIRST: uncomment occupation:
  ?person wdt:P31 wd:Q5;
           wdt:P106/wdt:P279* wd:Q28389; #uncomment for       screenwriter
  FILTER NOT EXISTS { ?person wdt:P570 ?date. } #person is alive
  
# SECOND: uncomment gender if applicable (for actor/actress):
#          wdt:P21 wd:Q6581097;    #male
#          wdt:P21 wd:Q6581072;    #female
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
kb = r.json()
for item in kb['results']['bindings']:
    people.add(unidecode.unidecode(item['personLabel']['value']))

In [ ]:
#print(tv_actors)

In [112]:
winners = {'cecil b. demille award': ['George', 'Clooney'],
    'best performance by an actress in a motion picture - drama': ['Julianne', 'Moore'],
    'best performance by an actor in a motion picture - drama': ['Eddie', 'Redmayne'],
    'best performance by an actress in a motion picture - comedy or musical': ['Amy', 'Adams'],
    'best performance by an actor in a motion picture - comedy or musical': ['Michael', 'Keaton'],
    'best performance by an actress in a supporting role in a motion picture': ['Patricia', 'Arquette'],
    'best performance by an actor in a supporting role in a motion picture': ['Simmons', 'J.', 'K.'],
    'best director - motion picture': ['Richard', 'Linklater'],
    'best screenplay - motion picture': ['Alejandro', 'Inarritu', 'Nicolas', 'Giacobone', 'Alexander', 'Dinelaris', 'Armando', 'Bo'],
    'best original score - motion picture': ['Johann', 'Johannsson'],
    'best performance by an actress in a television series - drama': ['Ruth', 'Wilson'],
    'best performance by an actor in a television series - drama': ['Kevin', 'Spacey'],
    'best performance by an actress in a television series - comedy or musical': ['Gina', 'Rodriguez'],
    'best performance by an actor in a television series - comedy or musical': ['Jeffrey', 'Tambor'],
    'best performance by an actress in a mini-series or motion picture made for television': ['Maggie', 'Gyllenhaal'],
    'best performance by an actor in a mini-series or motion picture made for television': ['Billy', 'Bob', 'Thorton'],
    'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television': ['Joanne', 'Froggatt'],
    'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television': ['Matt', 'Bomer']}

In [113]:
f = open('english.txt', 'r')
stop_words = f.read().splitlines()
#print(stopwords)

In [114]:
OFFICIAL_AWARDS_1315 = ['cecil b. demille award',
                        'best performance by an actress in a motion picture - drama', 
                        'best performance by an actor in a motion picture - drama',
                        'best performance by an actress in a motion picture - comedy or musical', 
                        'best performance by an actor in a motion picture - comedy or musical',
                        'best performance by an actress in a supporting role in a motion picture', 
                        'best performance by an actor in a supporting role in a motion picture', 
                        'best director - motion picture', 
                        'best screenplay - motion picture', 
                        'best original score - motion picture',
                        'best performance by an actress in a television series - drama', 
                        'best performance by an actor in a television series - drama',
                        'best performance by an actress in a television series - comedy or musical', 
                        'best performance by an actor in a television series - comedy or musical', 
                        'best performance by an actress in a mini-series or motion picture made for television', 
                        'best performance by an actor in a mini-series or motion picture made for television', 
                        'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 
                        'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']

In [115]:
def parseAward(award):
    award = re.split('\W+', award)
    award = [i for i in award if i not in stop_words]
    if 'motion' in award and 'picture' in award:
        award.remove('motion')
        award.remove('picture')
        award.append('motion picture')
    if 'mini' in award and 'series' in award:
        award.remove('mini')
        award.remove('series')
        award.append('miniseries')
    return award

In [116]:
def getCategoriesDict():
    categories_dict = dict()
    for a in OFFICIAL_AWARDS_1315:
        terms = parseAward(a)
        categories_dict[a] = terms

    return categories_dict

In [117]:
categories_dict = getCategoriesDict()

In [118]:
categories_dict

{'cecil b. demille award': ['cecil', 'demille', 'award'],
 'best performance by an actress in a motion picture - drama': ['performance',
  'actress',
  'drama',
  'motion picture'],
 'best performance by an actor in a motion picture - drama': ['performance',
  'actor',
  'drama',
  'motion picture'],
 'best performance by an actress in a motion picture - comedy or musical': ['performance',
  'actress',
  'comedy',
  'musical',
  'motion picture'],
 'best performance by an actor in a motion picture - comedy or musical': ['performance',
  'actor',
  'comedy',
  'musical',
  'motion picture'],
 'best performance by an actress in a supporting role in a motion picture': ['performance',
  'actress',
  'supporting',
  'role',
  'motion picture'],
 'best performance by an actor in a supporting role in a motion picture': ['performance',
  'actor',
  'supporting',
  'role',
  'motion picture'],
 'best director - motion picture': ['director', 'motion picture'],
 'best screenplay - motion picture'

In [214]:
def filter0(data, list1):
    synonyms = {
        'motion picture': ['motion picture', 'motion', 'picture', 'movie'],
        'television' : ['television', 'tv'],
        'miniseries' : ['mini-series', 'mini', 'series', 'miniseries']
    }
    
    result = []
    
    list1 = [i for i in list1 if i != 'performance' and i != 'role']

    for tweet in data:
        cond = True
        for i in list1:
            if i in synonyms:
                if all(j not in tweet.lower() for j in synonyms[i]):
                    cond = False
            elif i not in tweet.lower():
                cond = False
        if cond:
            result.append(tweet)

    return result

In [185]:
def extractNominees(data, list1):
    #print(data[0])
    result = []
       
    #tt = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=True)
    translator = str.maketrans('', '', string.punctuation)
    remove_terms = ['#goldenglobes', 'golden globes', '#goldenglobe', 'golden globe', 'goldenglobes', 'goldenglobe', 'rt', 'golden', 'globe', 'globes']    
    stop = remove_terms + list1
    
    for tweet in data:
        #print(tweet)
        tweet = re.sub("\d+", "", tweet) #strip nums
        tweet = re.sub(r'http\S+', '', tweet) #strip urls
        tweet = re.sub(r'#\S+', '', tweet) #strip hashtags
        tweet = tweet.translate(translator) #strip non-alphanumeric characters
        tweet = tweet.split() #tokenize
        tweet = [term for term in tweet if term.lower() not in stop_words] #remove stop words
        for i in stop:
            for j in tweet:
                if i.lower() in j.lower():
                    tweet.remove(j)
        result.append(tweet)
        
    #print(result[:20])
        
    grams = [];

    for tweet in result:
        if tweet:
            gram = list(nltk.everygrams(tweet, 2, 3))
            #print(bigram[:10])
            for g in gram:
                if len(g) == 2:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])):
                        grams.append(g)
                else:
                    if bool(re.match(r'\b[A-Z][a-z]+\b', g[0])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[1])) and bool(re.match(r'\b[A-Z][a-z]+\b', g[2])):
                        grams.append(g)

  
    fdist = nltk.FreqDist(grams)
    #print(fdist)

    try:
        names = fdist.most_common()
        #names = [' '.join(i[0]) for i in fdist.most_common()]
    except:
        names = "nothing here"
    
    return names

In [121]:
def getNominees():
    d = {}
    for award in OFFICIAL_AWARDS_1315:
        d["{0}".format(award)] = filter0(data, categories_dict[award])
            
    nominees = {}
    for award in OFFICIAL_AWARDS_1315:
        nominees["{0}".format(award)] = extractNominees(d[award], categories_dict[award])
    
    return nominees

In [215]:
import timeit
start_time = timeit.default_timer()
nominees = getNominees()
elapsed = timeit.default_timer() - start_time
print(elapsed)

213.69207202100006


In [190]:
nominees.keys()

dict_keys(['cecil b. demille award', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - drama', 'best performance by an actress in a motion picture - comedy or musical', 'best performance by an actor in a motion picture - comedy or musical', 'best performance by an actress in a supporting role in a motion picture', 'best performance by an actor in a supporting role in a motion picture', 'best director - motion picture', 'best screenplay - motion picture', 'best original score - motion picture', 'best performance by an actress in a television series - drama', 'best performance by an actor in a television series - drama', 'best performance by an actress in a television series - comedy or musical', 'best performance by an actor in a television series - comedy or musical', 'best performance by an actress in a mini-series or motion picture made for television', 'best performance by an actor in a mini-series or motion picture mad

In [204]:
def compareToKB(list1):
    for i in list1:
        j = ' '.join(i[0])
        if j in people:
            print(i)

In [126]:
#Julianne Moore – Still Alice as Dr. Alice Howland - 3132
#Jennifer Aniston – Cake as Claire Bennett -13
#Felicity Jones – The Theory of Everything as Jane Hawking - 6
#Rosamund Pike – Gone Girl as Amy Elliott-Dunne - 11
#Reese Witherspoon – Wild as Cheryl Strayed - 6

In [216]:
compareToKB(nominees['cecil b. demille award'])

(('George', 'Clooney'), 2440)
(('Julianna', 'Margulies'), 30)
(('Alejandro', 'Gonzalez', 'Inarritu'), 9)
(('Liza', 'Minnelli'), 5)
(('Michael', 'Keaton'), 5)
(('Meryl', 'Streep'), 3)
(('Jim', 'Carrey'), 2)
(('Jodie', 'Foster'), 2)
(('Gary', 'Busey'), 1)
(('Pia', 'Zadora'), 1)
(('Kevin', 'Spacey'), 1)
(('Matt', 'Damon'), 1)
(('Dakota', 'Fanning'), 1)
(('Angelina', 'Jolie'), 1)
(('Richard', 'Linklater'), 1)


In [218]:
compareToKB(nominees['best performance by an actress in a motion picture - drama'])

(('Julianne', 'Moore'), 3298)
(('Rosamund', 'Pike'), 22)
(('Jennifer', 'Aniston'), 14)
(('Patricia', 'Arquette'), 11)
(('Reese', 'Witherspoon'), 9)
(('Felicity', 'Jones'), 7)
(('Amy', 'Adams'), 7)
(('Eddie', 'Redmayne'), 7)
(('Shailene', 'Woodley'), 6)
(('Richard', 'Linklater'), 1)
(('George', 'Clooney'), 1)


In [219]:
compareToKB(nominees['best performance by an actor in a motion picture - drama'])

(('Eddie', 'Redmayne'), 3268)
(('Kevin', 'Spacey'), 415)
(('David', 'Oyelowo'), 14)
(('Benedict', 'Cumberbatch'), 13)
(('Jake', 'Gyllenhaal'), 10)
(('Michael', 'Keaton'), 8)
(('Steve', 'Carell'), 7)
(('Julianne', 'Moore'), 6)
(('Gwyneth', 'Paltrow'), 6)
(('George', 'Clooney'), 2)
(('Jennifer', 'Aniston'), 1)
(('Frank', 'Underwood'), 1)


In [220]:
compareToKB(nominees['best performance by an actress in a motion picture - comedy or musical'])

(('Amy', 'Adams'), 3511)
(('Gina', 'Rodriguez'), 86)
(('Emily', 'Blunt'), 7)
(('Patricia', 'Arquette'), 6)
(('Helen', 'Mirren'), 4)
(('Ricky', 'Gervais'), 4)
(('Julianne', 'Moore'), 3)
(('Michael', 'Keaton'), 1)
(('Quvenzhane', 'Wallis'), 1)
(('Meryl', 'Streep'), 1)
(('Lana', 'Parrilla'), 1)


In [221]:
compareToKB(nominees['best performance by an actor in a motion picture - comedy or musical'])

(('Michael', 'Keaton'), 2976)
(('Jeffrey', 'Tambor'), 65)
(('Amy', 'Adams'), 5)
(('Ralph', 'Fiennes'), 3)
(('Joaquin', 'Phoenix'), 2)
(('Christoph', 'Waltz'), 2)
(('Emily', 'Blunt'), 1)
(('Jamie', 'Foxx'), 1)
(('Wes', 'Anderson'), 1)


In [222]:
compareToKB(nominees['best director - motion picture'])

(('Richard', 'Linklater'), 3231)
(('Angelina', 'Jolie'), 14)
(('Ellar', 'Coltrane'), 14)
(('Wes', 'Anderson'), 9)
(('Patricia', 'Arquette'), 3)
(('Eddie', 'Redmayne'), 3)
(('David', 'Fincher'), 2)
(('Ethan', 'Hawke'), 2)
(('Justin', 'Lin'), 1)
(('Julianne', 'Moore'), 1)
(('Seth', 'Meyers'), 1)
(('Cary', 'Fukunaga'), 1)
(('Andrey', 'Zvyagintsev'), 1)


In [223]:
compareToKB(nominees['best screenplay - motion picture'])

(('Patricia', 'Arquette'), 25)
(('Kristen', 'Wiig'), 20)
(('Alejandro', 'Gonzalez', 'Inarritu'), 6)
(('Emma', 'Stone'), 3)
(('Michael', 'Keaton'), 3)
(('Richard', 'Linklater'), 2)
(('Tyler', 'Perry'), 2)
(('Michael', 'Bay'), 2)
(('Mike', 'Leigh'), 2)
(('Julianne', 'Moore'), 1)
(('Gillian', 'Flynn'), 1)
(('Judd', 'Apatow'), 1)
(('Michael', 'Mann'), 1)
(('Adam', 'Sandler'), 1)
(('Wes', 'Anderson'), 1)


In [224]:
compareToKB(nominees['best original score - motion picture'])

(('Hans', 'Zimmer'), 3)
(('Sienna', 'Miller'), 2)
(('Vince', 'Vaughn'), 2)
(('Kevin', 'Spacey'), 2)
(('John', 'Legend'), 2)
(('Emma', 'Stone'), 1)
(('Alexandre', 'Desplat'), 1)


In [225]:
compareToKB(nominees['best performance by an actress in a television series - drama'])

(('Viola', 'Davis'), 11)
(('Claire', 'Danes'), 7)
(('Chris', 'Pratt'), 4)
(('Lana', 'Parrilla'), 4)
(('Julianna', 'Margulies'), 3)
(('Robin', 'Wright'), 3)
(('Gina', 'Rodriguez'), 2)
(('Anna', 'Faris'), 2)
(('Kevin', 'Spacey'), 2)
(('Angela', 'Lansbury'), 1)
(('Maggie', 'Gyllenhaal'), 1)
(('Jessica', 'Lange'), 1)
(('Allison', 'Tolman'), 1)
(('Dominic', 'West'), 1)


In [136]:
# RUTH WILSON NOT IN KB EVEN THOUGH SHE IS AN ACTOR AND ALIVE

In [226]:
compareToKB(nominees['best performance by an actor in a television series - drama'])

(('Kevin', 'Spacey'), 4889)
(('Frank', 'Underwood'), 32)
(('Kate', 'Mara'), 8)
(('Clive', 'Owen'), 7)
(('Dominic', 'West'), 6)
(('Liev', 'Schreiber'), 6)
(('James', 'Spader'), 5)
(('Katherine', 'Heigl'), 3)
(('David', 'Duchovny'), 3)
(('Amy', 'Adams'), 3)
(('Billy', 'Bob', 'Thornton'), 1)
(('Matt', 'Bomer'), 1)
(('Jeffrey', 'Tambor'), 1)
(('Deepika', 'Padukone'), 1)


In [227]:
compareToKB(nominees['best performance by an actress in a television series - comedy or musical'])

(('Gina', 'Rodriguez'), 1943)
(('Kerry', 'Washington'), 2)
(('Chloe', 'Bennet'), 2)
(('Bryan', 'Cranston'), 1)
(('Lena', 'Dunham'), 1)
(('Edie', 'Falco'), 1)
(('Taylor', 'Schilling'), 1)
(('Amy', 'Adams'), 1)


In [228]:
compareToKB(nominees['best performance by an actor in a television series - comedy or musical'])

(('Jeffrey', 'Tambor'), 1247)
(('Matt', 'Bomer'), 3)
(('Ricky', 'Gervais'), 3)
(('Jane', 'Fonda'), 2)
(('Katie', 'Holmes'), 1)
(('Seth', 'Meyers'), 1)
(('Lily', 'Tomlin'), 1)
(('Gina', 'Rodriguez'), 1)


In [229]:
compareToKB(nominees['best performance by an actress in a mini-series or motion picture made for television'])

(('Maggie', 'Gyllenhaal'), 2160)
(('Gina', 'Rodriguez'), 90)
(('Adrien', 'Brody'), 3)
(('Kate', 'Beckinsale'), 2)
(('Uzo', 'Aduba'), 1)
(('Kathy', 'Bates'), 1)
(('Dakota', 'Johnson'), 1)
(('Amy', 'Adams'), 1)
(('Jessica', 'Lange'), 1)
(('Allison', 'Tolman'), 1)
(('Judith', 'Light'), 1)


In [230]:
compareToKB(nominees['best performance by an actor in a mini-series or motion picture made for television'])

(('Billy', 'Bob', 'Thornton'), 2142)
(('Matt', 'Bomer'), 1357)
(('Kevin', 'Spacey'), 252)
(('Jeffrey', 'Tambor'), 5)
(('Jeremy', 'Renner'), 2)
(('Mark', 'Ruffalo'), 2)
(('Jon', 'Voight'), 2)
(('James', 'Spader'), 1)
(('Woody', 'Harrelson'), 1)
(('Katie', 'Holmes'), 1)
(('Seth', 'Meyers'), 1)
(('Alan', 'Cumming'), 1)
(('Colin', 'Hanks'), 1)
(('Frank', 'Underwood'), 1)


In [231]:
compareToKB(nominees['best performance by an actress in a supporting role in a series, mini-series or motion picture made for television'])

(('Maggie', 'Gyllenhaal'), 7)
(('Uzo', 'Aduba'), 1)
(('Kathy', 'Bates'), 1)
(('Dakota', 'Johnson'), 1)


In [232]:
compareToKB(nominees['best performance by an actor in a supporting role in a series, mini-series or motion picture made for television'])

(('Matt', 'Bomer'), 1246)
(('Billy', 'Bob', 'Thornton'), 2)
(('Jon', 'Voight'), 2)
(('Alan', 'Cumming'), 1)
(('Colin', 'Hanks'), 1)
(('Mark', 'Ruffalo'), 1)
